# MODELIZACIÓN PARA CLASIFICACIÓN

## IMPORTAR PAQUETES

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import roc_auc_score

from sklearn.metrics import classification_report


#Autocompletar rápido
%config IPCompleter.greedy=True

#Desactivar la notación científica
pd.options.display.float_format = '{:.2f}'.format

#Desactivar los warnings
import warnings
warnings.filterwarnings("ignore")

## IMPORTAR LOS DATOS

Sustituir la ruta del proyecto.

In [2]:
ruta_proyecto = r'C:\Users\iagzg\Desktop\D4FB\D4FB\EstructuraDirectorio\03_MACHINE_LEARNING\06-MACHINE LEARNING PROJECT FRAMEWORK\CASOS\04_BANK\\'

Nombres de los ficheros de datos.

In [3]:
nombre_x = 'x_preseleccionado.pickle'
nombre_y = 'y_preseleccionado.pickle'

Cargar los datos.

In [4]:
x = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_x)
y = pd.read_pickle(ruta_proyecto + '/02_Datos/03_Trabajo/' + nombre_y)

## MODELIZAR

### Reservar el dataset de validacion

In [5]:
train_x,val_x,train_y,val_y = train_test_split(x,y,test_size=0.3)

### Crear el pipe y el diccionario de algorimos, parámetros y valores a testar

Modificar para dejar solo los algoritmos que se quieran testar.

Modificar los parámetros.

In [6]:
pipe = Pipeline([('algoritmo',RandomForestClassifier())])

grid = [{'algoritmo': [LogisticRegression()],
         'algoritmo__n_jobs': [-1],
         'algoritmo__solver': ['saga'],
         'algoritmo__penalty': ['elasticnet', 'l1', 'l2', 'none'],
         'algoritmo__C': [0,0.25,0.5,0.75,1]},
        
        {'algoritmo': [RandomForestClassifier()],
         'algoritmo__n_jobs': [-1],
         'algoritmo__max_depth': [5,10,15],
         'algoritmo__n_estimators': [50,100,200]},
        
        {'algoritmo': [XGBClassifier()],
         'algoritmo__n_jobs': [-1],
         'algoritmo__verbosity': [0],#para que no salgan warnings
         'algoritmo__learning_rate': [0.01,0.025,0.05,0.1],
         'algoritmo__max_depth': [5,10,20],
         'algoritmo__reg_alpha': [0,0.1,0.5,1],
         'algoritmo__reg_lambda': [0.01,0.1,1],
         'algoritmo__n_estimators': [100,500,1000]},
        
        {'algoritmo': [HistGradientBoostingClassifier()],
         'algoritmo__learning_rate': [0.01,0.025,0.05,0.1],
         'algoritmo__max_iter': [50,100,200],
         'algoritmo__max_depth': [5,10,20],
         'algoritmo__min_samples_leaf': [500],
         'algoritmo__scoring': ['roc_auc'],
         'algoritmo__l2_regularization': [0,0.25,0.5,0.75,1]}
       ]

### Optimizar los hiper parámetros

Elegir si se quiere usar grid search o random search.

Comentar la opción que no se vaya a usar.

####  Con grid search

In [7]:
grid_search = GridSearchCV(estimator= pipe, 
                           param_grid = grid, 
                           cv = 3, 
                           scoring = 'roc_auc',
                           verbose = 0,
                           n_jobs = -1)

modelo = grid_search.fit(train_x,train_y)

pd.DataFrame(grid_search.cv_results_).sort_values(by = 'rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algoritmo,param_algoritmo__C,param_algoritmo__n_jobs,param_algoritmo__penalty,param_algoritmo__solver,param_algoritmo__max_depth,...,param_algoritmo__max_iter,param_algoritmo__min_samples_leaf,param_algoritmo__scoring,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
22,0.57,0.06,0.12,0.04,RandomForestClassifier(),NaN,-1,NaN,NaN,5,...,NaN,NaN,NaN,"{'algoritmo': RandomForestClassifier(), 'algor...",0.72,0.73,0.72,0.72,0.01,1
20,0.10,0.01,0.05,0.04,RandomForestClassifier(),NaN,-1,NaN,NaN,5,...,NaN,NaN,NaN,"{'algoritmo': RandomForestClassifier(), 'algor...",0.72,0.73,0.72,0.72,0.00,2
21,0.24,0.02,0.04,0.00,RandomForestClassifier(),NaN,-1,NaN,NaN,5,...,NaN,NaN,NaN,"{'algoritmo': RandomForestClassifier(), 'algor...",0.72,0.73,0.72,0.72,0.01,3
148,0.25,0.05,0.02,0.00,"XGBClassifier(base_score=None, booster=None, c...",NaN,-1,NaN,NaN,5,...,NaN,NaN,NaN,"{'algoritmo': XGBClassifier(base_score=None, b...",0.71,0.72,0.72,0.72,0.01,4
147,0.23,0.05,0.02,0.00,"XGBClassifier(base_score=None, booster=None, c...",NaN,-1,NaN,NaN,5,...,NaN,NaN,NaN,"{'algoritmo': XGBClassifier(base_score=None, b...",0.71,0.72,0.72,0.72,0.01,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,0.00,0.00,0.00,0.00,LogisticRegression(),0.75,-1,none,saga,NaN,...,NaN,NaN,NaN,"{'algoritmo': LogisticRegression(), 'algoritmo...",NaN,NaN,NaN,NaN,NaN,630
16,0.00,0.00,0.00,0.00,LogisticRegression(),1,-1,elasticnet,saga,NaN,...,NaN,NaN,NaN,"{'algoritmo': LogisticRegression(), 'algoritmo...",NaN,NaN,NaN,NaN,NaN,630
19,0.00,0.00,0.00,0.00,LogisticRegression(),1,-1,none,saga,NaN,...,NaN,NaN,NaN,"{'algoritmo': LogisticRegression(), 'algoritmo...",NaN,NaN,NaN,NaN,NaN,630
8,0.00,0.00,0.00,0.00,LogisticRegression(),0.50,-1,elasticnet,saga,NaN,...,NaN,NaN,NaN,"{'algoritmo': LogisticRegression(), 'algoritmo...",NaN,NaN,NaN,NaN,NaN,630


Hay que decir que la Regresión Logística en estos datos funciona mal, ya que a pesar de haber hecho el reescalado y preselección de variables, no hemos aplicado técnicas de normalización. Otros algoritmos no son tan dependientes de estos requisitos, como puede ser el Random Forest.

## EVALUAR

### Predecir sobre validación

In [8]:
pred = modelo.best_estimator_.predict_proba(val_x)[:, 1]

### Evaluar sobre validación

In [9]:
roc_auc_score(val_y, pred)

0.7386245424436844

### Examinar el mejor modelo

In [10]:
modelo.best_estimator_

Pipeline(steps=[('algoritmo',
                 RandomForestClassifier(max_depth=5, n_estimators=200,
                                        n_jobs=-1))])

El mejor modelo es un Random Forest. Más adelante lo incluiremos en la fase de producción.